In [1]:
# %pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
# %pip install -q datasets bitsandbytes einops wandb

In [1]:
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    logging,
)
from typing import List
import torch
from torch import cuda, bfloat16
from datasets import load_dataset
import os
 
import matplotlib.pyplot as plt
import matplotlib as mpl
# import seaborn as sns
from pylab import rcParams
# from trl import SFTTrainer
 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [2]:
model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_8bit=True,
)


# Need auth token for these
hf_auth = os.environ.get('hf_token')

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

model.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded on cuda:0


In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)
 
# tokenizer.pad_token_id = (0)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [4]:
data = load_dataset("json", data_files="/home/hb/LLM-research/finetuning_dataset/BGP_knowledge.json")
data["train"]

Found cached dataset json (/home/hb/.cache/huggingface/datasets/json/default-954e161ec1be7a92/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 14866
})

In [5]:
CUTOFF_LEN = 512

def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
 
 
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
 
    result["labels"] = result["input_ids"].copy()
 
    return result
 
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [6]:
train_val = data["train"].train_test_split(
    test_size=1400, shuffle=True, seed=42
)
train_data = (
    train_val["train"].map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].map(generate_and_tokenize_prompt)
)

Loading cached split indices for dataset at /home/hb/.cache/huggingface/datasets/json/default-954e161ec1be7a92/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-bda3c556bbad67b1.arrow and /home/hb/.cache/huggingface/datasets/json/default-954e161ec1be7a92/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-64ebe0635b6e7740.arrow
Loading cached processed dataset at /home/hb/.cache/huggingface/datasets/json/default-954e161ec1be7a92/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-6c598db7cfe4c632.arrow


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [7]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
 
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

output_dir = "./llama_BGP_10"
per_device_train_batch_size = 4
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 500
logging_steps = 200
learning_rate = 1e-4
max_grad_norm = 0.3
max_steps = 20000
warmup_ratio = 0.05
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    # save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [8]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, return_tensors="pt", padding=True
)

In [9]:
from trl import SFTTrainer

max_seq_length = None

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=peft_config,
    dataset_text_field="output",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/hb/myenv/lib/python3.8/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/hb/myenv/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Loading cached processed dataset at /home/hb/.cache/huggingface/datasets/json/default-954e161ec1be7a92/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-a822c1153aac6c48.arrow


Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

In [11]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Explain what BGP blackholing activity is"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/hb/myenv/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/hb/myenv/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<s>[INST] Explain what BGP blackholing activity is [/INST]  BGP blackholing activity is a malicious practice in which an attacker injects false BGP routes into the network in order to redirect traffic away from its intended destination. This can be used to disrupt services, intercept traffic, or even launch denial-of-service attacks. BGP blackholing activity can be difficult to detect, as it often involves subtle changes to existing routes or the introduction of new routes. To protect against BGP blackholing activity, network operators should implement security measures such as route filtering,


In [12]:
new_model = "llama2-13b-BGP_10"

trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('llama2-13b-BGP_10/tokenizer_config.json',
 'llama2-13b-BGP_10/special_tokens_map.json',
 'llama2-13b-BGP_10/tokenizer.model',
 'llama2-13b-BGP_10/added_tokens.json',
 'llama2-13b-BGP_10/tokenizer.json')

: 

### True-False

In [ ]:
import json
import re
from transformers import pipeline

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_true_false.json" 

# Load questions and answers from JSON file
with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

# Initialize text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

llama2_answers = []

# Evaluate generated answers
correct_answers = 0
total_questions = len(questions_data)

for q_dict in questions_data:
    question = q_dict["question"]
    answer = q_dict["answer"]

    # Generate answer using the language model
    prompt = f"[INST] {question} [/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    # print(generated_answer)
    llama2_answers.append(generated_answer)

# Extract ground truth answers
ground_truth = [q_dict["answer"] for q_dict in questions_data]

print(ground_truth)

for ground_answer, generated_answer in zip(ground_truth, llama2_answers):
    generated_answer = re.search(r'\[/INST\]\s+(.*?)\.', generated_answer).group(1).strip()
    print(f"Generated Answer: {generated_answer}")
    
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print("Incorrect")

accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")


In [ ]:
import json
import re

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_fill_the_blank.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers_fb = []
filled_answers = []

n_question = 0
for q_dict in questions_data:
    question = q_dict["question"]
    answer = q_dict["answer"]

    # Generate answer using the language model
    prompt = f"[INST]{question} \n Fill the blank: \n [/INST]"
    generated_answer = pipe(prompt)[0]['generated_text']
    # print(f"Output: {generated_answer}")
    
    generated_answer = re.search(r'\[/INST\]\s+(.*)', generated_answer).group(1).strip()
    if "Sure" in generated_answer:
        generated_lines = generated_answer.split('\n')
        if len(generated_lines) > 1:
            generated_answer = '\n'.join(generated_lines[1:])  # Skip the first line
    generated_answers_fb.append(generated_answer)
    print(f"Generated answer: {generated_answer}")
    # Find the position of the placeholder in the first string
    placeholder_position = question.find("________")

    # Extract the content filled in the placeholder
    filled_content = generated_answer[placeholder_position:placeholder_position + len("autonomous systems (ASes)")]
    filled_answers.append(filled_content)
    n_question += 1
    print(f"Filled content: {filled_content}")
    print(f"---------------------{n_question}-----------------------------")

In [ ]:
import json
import re

input_file_path = "/home/hb/fine-tuning-alpaca/evaluation/bgp_test_multiple_choice.json" 

with open(input_file_path, "r") as input_file:
    questions_data = json.load(input_file)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)

generated_answers = []
ground_truth = [q_dict["answer"] for q_dict in questions_data]
n_question = 0

for q_dict in questions_data:
    question = q_dict["question"]
    options = q_dict["options"]
    
    # Construct prompt with question and options
    prompt = f"Choose the correct answer: \n{question}\n" + "\n".join(options)
    # print(prompt)
    
    # Generate answer using the language model
    generated_answer = pipe(prompt)[0]['generated_text']
    print(f"Generated Answer: {generated_answer}")
    
    # Parse the generated answer
    answer_lines = generated_answer.splitlines()
    parsed_answer = ""
    for line in answer_lines:
        if line.startswith("Answer: ") or line.startswith("Correct answer: "):
            parsed_answer = re.search(r'\b(\w)\)', line).group(1)
            print(f"Parsed_answer: {parsed_answer}")
            break
    generated_answers.append(parsed_answer)
    n_question += 1
    print(f"----------------{n_question}-----------------")
print(generated_answers)


total_questions = len(questions_data)
correct_answers = 0

for ground_answer, generated_answer in zip(ground_truth, generated_answers):
    if generated_answer.lower() == ground_answer.lower():
        print("Correct")
        correct_answers += 1
    else:
        print("Incorrect")
        print(f"LLaMA answer: {generated_answer}. Correct answer: {ground_answer}")    
accuracy = (correct_answers / total_questions) * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Correct answers: {correct_answers}")
print(f"Incorrect answers: {total_questions - correct_answers}")

# Saving the model

In [2]:
from peft import LoraConfig, PeftModel
model_id = 'meta-llama/Llama-2-13b-chat-hf'
new_model = "llama2-13b-BGP_10"

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto',
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
model.push_to_hub('hyonbokan/BGP-llama_20k-constant')
tokenizer.push_to_hub('hyonbokan/BGP-llama_20k-constant')

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hyonbokan/BGP-llama_20k-constant/commit/4189b242574f1db0a6a190e157752d00b2875d0e', commit_message='Upload tokenizer', commit_description='', oid='4189b242574f1db0a6a190e157752d00b2875d0e', pr_url=None, pr_revision=None, pr_num=None)